In [5]:
!pip install transformers
!pip install nltk
nltk.download('punkt')

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [1]:
import torch
from transformers import BertTokenizer, BertModel
import nltk

# Load pre-trained BERT model
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased')

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/455k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/420M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [2]:
def summarize(text, k=3):
    # Tokenize text into sentences
    sentences = nltk.sent_tokenize(text)
    
    # Encode sentences using BERT tokenizer and obtain sentence embeddings
    encoded_sentences = tokenizer.batch_encode_plus(sentences, padding=True, truncation=True, return_tensors='pt')
    with torch.no_grad():
        model_output = model(**encoded_sentences)
        sentence_embeddings = model_output.last_hidden_state[:, 0, :]
    
    # Compute average sentence embedding
    avg_embedding = sentence_embeddings.mean(dim=0)
    
    # Compute cosine similarity between sentence embeddings and average embedding
    similarities = torch.nn.functional.cosine_similarity(sentence_embeddings, avg_embedding, dim=1)
    
    # Select top-k sentences with highest similarity scores as summary
    summary = [sentences[i] for i in similarities.argsort(descending=True)[:k]]
    
    return ' '.join(summary)


In [3]:
text = """
The Great Barrier Reef is the world's largest coral reef system composed of over 2,900 individual reefs and 900 islands 
stretching for over 2,300 kilometres (1,400 mi) over an area of approximately 344,400 square kilometres (133,000 sq mi). 
The reef is located in the Coral Sea, off the coast of Queensland, Australia. The Great Barrier Reef can be seen from outer space 
and is the world's biggest single structure made by living organisms. This reef structure is composed of and built by billions of 
tiny organisms, known as coral polyps. It supports a wide diversity of life and was selected as a World Heritage Site in 1981. 
CNN has called it one of the seven natural wonders of the world.
"""

summary = summarize(text, k=2)
print(summary)


The Great Barrier Reef can be seen from outer space 
and is the world's biggest single structure made by living organisms. This reef structure is composed of and built by billions of 
tiny organisms, known as coral polyps.


# Rogue score comparision code


In [4]:
!pip install sentencepiece

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [17]:
!pip install transformers
!pip install rouge

from transformers import TFT5ForConditionalGeneration, T5Tokenizer
from rouge import Rouge

# Load the T5 model and tokenizer
model = TFT5ForConditionalGeneration.from_pretrained('t5-base')
tokenizer = T5Tokenizer.from_pretrained('t5-base')

# Define the input and output texts
input_text = "The input text is a little bit more complicated."
output_text = "The output text is also a little bit more complicated."

# Generate summaries using the T5 model
generated_summary1 = model.generate(
    input_ids=tokenizer.encode(input_text, return_tensors="tf"),
    max_length=50,
    num_beams=5,
    no_repeat_ngram_size=2,
    early_stopping=True
)
generated_summary2 = model.generate(
    input_ids=tokenizer.encode(output_text, return_tensors="tf"),
    max_length=50,
    num_beams=5,
    no_repeat_ngram_size=2,
    early_stopping=True
)

# Decode the generated summaries
generated_summary1 = tokenizer.decode(generated_summary1[0], skip_special_tokens=True)
generated_summary2 = tokenizer.decode(generated_summary2[0], skip_special_tokens=True)

# Define the reference summaries
reference_summaries = [
    "The input text goes here.",
    "The output text goes here."
]

# Compute ROUGE scores for the two models
rouge = Rouge()
scores1 = rouge.get_scores([generated_summary1], reference_summaries, avg=True)
scores2 = rouge.get_scores([generated_summary2], reference_summaries, avg=True)

print("Model 1 scores:", scores1)
print("Model 2 scores:", scores2)


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


All model checkpoint layers were used when initializing TFT5ForConditionalGeneration.

All the layers of TFT5ForConditionalGeneration were initialized from the model checkpoint at t5-base.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFT5ForConditionalGeneration for predictions without further training.


AssertionError: ignored

In [20]:
!pip install rouge

from rouge import Rouge

# Define the reference summaries
reference_summaries = [
    "John likes to watch movies. Mary likes movies too.",
    "John also likes to watch football games.",
    "Mary hates football."
]

# Define the generated summaries
generated_summary1 = "John and Mary enjoy watching movies together."
generated_summary2 = "John enjoys watching football games."

# Convert the reference summaries to a list of lists of sentences
reference_summaries = [summary.split('. ') for summary in reference_summaries]

# Compute ROUGE scores for the two models
rouge = Rouge()
scores1 = rouge.get_scores([generated_summary1], reference_summaries, avg=True)
scores2 = rouge.get_scores([generated_summary2], reference_summaries, avg=True)

# Print the ROUGE-1 F1 scores for the two models
print("ROUGE-1 F1 scores:")
print(f"Model 1: {scores1['rouge-1']['f']:.2f}")
print(f"Model 2: {scores2['rouge-1']['f']:.2f}")


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


AssertionError: ignored

In [13]:
from transformers import T5Tokenizer, TFT5ForConditionalGeneration
from rouge import Rouge

# Load T5 tokenizer and model
tokenizer = T5Tokenizer.from_pretrained('t5-base')
model = TFT5ForConditionalGeneration.from_pretrained('t5-base')

# Define input text
input_text = "Input text goes here. The input text is a little bit more complicated."

# Generate summaries using T5 model
generated_summaries1 = model.generate(
    input_ids=tokenizer.encode(input_text, return_tensors="tf"),
    max_length=50,
    num_beams=4,
    early_stopping=True
)

generated_summaries2 = model.generate(
    input_ids=tokenizer.encode(input_text, return_tensors="tf"),
    max_length=50,
    num_beams=4,
    early_stopping=True
)

# Decode generated summaries
generated_summaries1 = [tokenizer.decode(g, skip_special_tokens=True) for g in generated_summaries1.numpy()]
generated_summaries2 = [tokenizer.decode(g, skip_special_tokens=True) for g in generated_summaries2.numpy()]

# Define reference summaries
reference_summaries = ["The input text is a bit complicated.",
                       "The input text is a bit complex and needs to be simplified."]
print(len(generated_summaries1))
print(len(reference_summaries))

All model checkpoint layers were used when initializing TFT5ForConditionalGeneration.

All the layers of TFT5ForConditionalGeneration were initialized from the model checkpoint at t5-base.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFT5ForConditionalGeneration for predictions without further training.


1
2


1

In [16]:
from transformers import T5Tokenizer, T5ForConditionalGeneration
from rouge import Rouge

# Initialize the tokenizer and model
tokenizer = T5Tokenizer.from_pretrained('t5-base')
model = T5ForConditionalGeneration.from_pretrained('t5-base')

# Set the maximum length for the generated summaries
max_length = 50

# Define the input text and reference summaries
input_text = [
    "Scientists have discovered a new species of dinosaur in South America.",
    "The stock market reached record highs last week, buoyed by positive economic indicators.",
    "In a surprise announcement, the president has resigned from office, citing personal reasons."
]
reference_summaries = [
    "A new species of dinosaur has been found in South America.",
    "The stock market hit record highs thanks to positive economic indicators.",
    "The president has resigned from office due to personal reasons."
]

# Generate the summaries using the model and tokenizer
generated_summaries = []
for text in input_text:
    input_ids = tokenizer.encode(text, return_tensors='pt', max_length=max_length, truncation=True)
    summary_ids = model.generate(input_ids)
    summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
    generated_summaries.append(summary)

# Compute ROUGE scores for the two models
rouge = Rouge()
scores = rouge.get_scores(generated_summaries, reference_summaries)

# Print the ROUGE scores
print(scores)


TypeError: ignored

In [20]:
from transformers import T5Tokenizer, T5ForConditionalGeneration
import torch

# Initialize the tokenizer and model
tokenizer = T5Tokenizer.from_pretrained('t5-base', model_max_length=512)
model = T5ForConditionalGeneration.from_pretrained('t5-base')

# Example text
text = "The quick brown fox jumps over the lazy dog."

# Encode the text and generate summary
inputs = tokenizer.encode("summarize: " + text, return_tensors="pt", max_length=512, truncation=True)
outputs = model.generate(inputs, max_length=50, min_length=10, length_penalty=2.0, num_beams=4, early_stopping=True)
generated_summary = tokenizer.decode(outputs[0], skip_special_tokens=True)

print("Original text: ", text)
print("Generated summary: ", generated_summary)


TypeError: ignored

In [21]:
from transformers import T5Tokenizer, T5ForConditionalGeneration
from rouge import Rouge

# Download the tokenizer
tokenizer = T5Tokenizer.from_pretrained('t5-base')

# Initialize the model
model = T5ForConditionalGeneration.from_pretrained('t5-base')

# Define the input text and the reference summaries
input_text = "The quick brown fox jumps over the lazy dog."
reference_summaries = ["A quick brown fox jumps over the lazy dog.",
                       "A quick fox jumps over the lazy dog.",
                       "A brown fox jumps over the dog."]

# Generate the summaries using the model
generated_summaries = []
for summary in reference_summaries:
    input_ids = tokenizer.encode(summary, return_tensors='pt', max_length=512, truncation=True)
    output_ids = model.generate(input_ids, max_length=50)
    generated_summary = tokenizer.decode(output_ids[0], skip_special_tokens=True)
    generated_summaries.append(generated_summary)

# Compute ROUGE scores for the model
rouge = Rouge()
scores = rouge.get_scores(generated_summaries, reference_summaries, avg=True)

print(scores)


TypeError: ignored

In [23]:
from transformers import T5Tokenizer

tokenizer = T5Tokenizer.from_pretrained('t5-base')


TypeError: ignored

In [24]:
!pip uninstall transformers
!pip install transformers==4.11.2
!pip uninstall tokenizers
!pip install tokenizers==0.10.3


Found existing installation: transformers 4.27.3
Uninstalling transformers-4.27.3:
  Would remove:
    /usr/local/bin/transformers-cli
    /usr/local/lib/python3.9/dist-packages/transformers-4.27.3.dist-info/*
    /usr/local/lib/python3.9/dist-packages/transformers/*
Proceed (Y/n)? y
  Successfully uninstalled transformers-4.27.3
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 26.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 58.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 880.6/880.6 KB 52.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for sacremoses: filename=sacremoses-0.0.53-py3-none-any.whl size=895259 sha256=bcd12261c3123b26b363d88e3b5e2527af45ff4d7cb147202c1a9db679c0403a
  Stored in directory: /root/.cache/pip/wheels/12/1c/3d/46cf06718d63a32ff798a89594b61e7f345ab6b36d909ce033
Successfully built sac

Found existing installation: tokenizers 0.10.3
Uninstalling tokenizers-0.10.3:
  Would remove:
    /usr/local/lib/python3.9/dist-packages/tokenizers-0.10.3.dist-info/*
    /usr/local/lib/python3.9/dist-packages/tokenizers/*
Proceed (Y/n)? y
  Successfully uninstalled tokenizers-0.10.3
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached tokenizers-0.10.3-cp39-cp39-manylinux_2_5_x86_64.manylinux1_x86_64.manylinux_2_12_x86_64.manylinux2010_x86_64.whl (3.3 MB)


In [25]:
y


NameError: ignored

In [8]:
!pip install rouge

from rouge import Rouge
from transformers import pipeline

# Define the two models for comparison
model1 = pipeline('summarization', model='t5-small', tokenizer='t5-small', framework='tf')
model2 = pipeline('summarization', model='t5-base', tokenizer='t5-base', framework='tf')

# Define the reference summaries for the input text
reference_summaries = [
    'A summary of the text goes here.',
    'Another summary of the text goes here.'
]

# Define the input text for summarization
input_text = 'The input text goes here.'

# Generate summaries using the two models
generated_summaries1 = [model1(input_text, max_length=50, min_length=10, num_beams=2, early_stopping=True)]
generated_summaries2 = [model2(input_text, max_length=50, min_length=10, num_beams=2, early_stopping=True)]

# Print the generated summaries
print('Model 1 generated summaries:', generated_summaries1)
print('Model 2 generated summaries:', generated_summaries2)

# Compute ROUGE scores for the two models
rouge = Rouge()
scores1 = []
scores2 = []
for i in range(len(reference_summaries)):
    score1 = rouge.get_scores(generated_summaries1[i][0]['summary_text'], reference_summaries[i])
    scores1.append(score1[0])
    score2 = rouge.get_scores(generated_summaries2[i][0]['summary_text'], reference_summaries[i])
    scores2.append(score2[0])

# Print the ROUGE scores for the two models
print('Model 1 ROUGE scores:', scores1)
print('Model 2 ROUGE scores:', scores2)


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


All model checkpoint layers were used when initializing TFT5ForConditionalGeneration.

All the layers of TFT5ForConditionalGeneration were initialized from the model checkpoint at t5-small.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFT5ForConditionalGeneration for predictions without further training.
All model checkpoint layers were used when initializing TFT5ForConditionalGeneration.

All the layers of TFT5ForConditionalGeneration were initialized from the model checkpoint at t5-base.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFT5ForConditionalGeneration for predictions without further training.
Your max_length is set to 50, but you input_length is only 9. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=50)
Your max_length is set to 50, but you input_length is only 9. You might consider decreasing max_length manually, e.g. summarizer

Model 1 generated summaries: [[{'summary_text': 'input text goes here . the input text is a little bit more complicated .'}]]
Model 2 generated summaries: [[{'summary_text': 'the input text goes here. the text goes to the input.'}]]


IndexError: ignored